# V 0.0

This is next "version" of [simple finetuning](https://www.kaggle.com/code/yannchikk/bart-large-cnn-dialoguesum-booksum-full-finetuning). There i try train the bart-large-cnn by LoRA finetuning method with PEFT lib.

There i going to train LoRA with processed text and text without processing and make Model Soup from this two versions.

In this Notebook:

- ## Train original '[bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)'

- ## Text processing

- ## Using peft lib for add LoRA layers to model

In [ ]:
class Config:

    max_length = 1024
    target_max_length = 512

    epochs = 2

    batch_size = 4

#     model_preset_trained = "doublecringe123/bardt-large-cnn-dialoguesum-booksum"

    try:
        model_preset = model_preset_trained
    except:
        model_preset = "facebook/bart-large-cnn"


    inp = 'input_content'
    target = 'target'

    save_limits = 1

cfg = Config()

# Load and Import Libraries

In [ ]:
! pip install --upgrade pip
! pip install -q transformers[torch]
! pip install -q -U transformers datasets==2.18.0 evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 971.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.1 

In [ ]:
! pip install -q peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
# load datasets from hugging face
from datasets import load_dataset, concatenate_datasets

import pandas as pd

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
! pip install -q nltk

import nltk

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

from nltk.tokenize import word_tokenize

from nltk.stem import SnowballStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load datasets



In [ ]:
dialoguesum = load_dataset(path = "knkarthick/dialogsum")

booksum = load_dataset(path = "kmfoda/booksum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print("Dialoguesum columns:")
print(dialoguesum['train'].column_names)

print("\nBooksum columns:")
print(booksum['train'].column_names)

Dialoguesum columns:
['id', 'dialogue', 'summary', 'topic']

Booksum columns:
['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length']


In [ ]:
# prompt: rename 'dialogue', 'summary' column of dialoguesum to 'content' and 'target'. 'chapter' and 'summary_text' of booksum to ''content'' and 'target'

dialoguesum = dialoguesum.rename_column("dialogue", cfg.inp)
dialoguesum = dialoguesum.rename_column("summary", cfg.target)

booksum = booksum.rename_column("chapter", cfg.inp)
booksum = booksum.rename_column("summary_text", cfg.target)

print("Dialoguesum columns:")
print(dialoguesum['train'].column_names)

print("\nBooksum columns:")
print(booksum['train'].column_names)


Dialoguesum columns:
['id', 'input_content', 'target', 'topic']

Booksum columns:
['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'input_content', 'chapter_length', 'summary_name', 'summary_url', 'target', 'summary_analysis', 'summary_length', 'analysis_length']


# Prepare Dataset

In [ ]:
# prompt: remove all columns except 'input_text' and 'target' and concat the datasets

dialoguesum = dialoguesum.remove_columns([i for i in dialoguesum['train'].column_names if ((i == cfg.target) or (i == cfg.inp)) == False])
booksum = booksum.remove_columns([i for i in booksum['train'].column_names if ((i != cfg.target) or (i != cfg.inp)) == False])

In [ ]:
dataset_train = concatenate_datasets([dialoguesum['train'], booksum['train']])

dataset_val = concatenate_datasets([dialoguesum['validation'], booksum['validation']])

dataset_test = concatenate_datasets([dialoguesum['test'], booksum['test']])

# Load model and tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model_preset)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_preset)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

# Creating LoRA Model with PEFT

PEFT refers to a group of techniques that enable efficient adaptation of large language models (LLMs) to specific tasks or domains. It involves fine-tuning only a small subset of parameters in the LLM, rather than modifying the entire model. This approach offers several advantages:

I knew about PEFT and LoRA models buildin from [this notebook](https://www.kaggle.com/code/ajinkyabhandare2002/fine-tune-flan-t5-base-for-chat-with-peft-lora#Setup-the-PEFT/LoRA-model-for-Fine-Tuning)

In [ ]:
lora_conf = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout = 0.09,
    task_type = TaskType.CAUSAL_LM,
)

In [ ]:
lora_model = get_peft_model(model=model, peft_config=lora_conf)

lora_model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.28950542822677927


# Prepare text

In [ ]:
stemmer = SnowballStemmer(language='english', ignore_stopwords = False)

In [ ]:

def text_remove_stopwords(seq) -> list:
    """"Assuming that seq is tokenized text"""
    return [t for t in seq if t not in eng_stopwords]

def text_stemming(seq) -> list:
    """"Assuming that seq is tokenized text"""

    return [stemmer.stem(word) for word in seq]

def text_process(text, fn_list = None) -> str:
    tokens = word_tokenize(" ".join(text.split()))

    if fn_list == None:
        fn_list = [text_remove_stopwords, text_stemming]

    for fn in fn_list:
        tokens = fn(tokens)

    return " ".join(tokens)

# Preprocess Datasets

In [ ]:
def preprocess_function(examples):
    inputs = [text_process(doc) for doc in examples[cfg.inp]]
    model_inputs = tokenizer(inputs, max_length=cfg.max_length, truncation=True)

    labels = tokenizer(text_target=examples[cfg.target], max_length=cfg.target_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset_train.map(preprocess_function, batched=True)
tokenized_val = dataset_val.map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/22060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1984 [00:00<?, ? examples/s]

Map:   0%|          | 0/2931 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=cfg.model_preset)

# Define Metrics

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Define training arguments

In [ ]:
try: 
    import wandb
    wandb.init(mode='disabled')
except: 
    print("Weights and biases not installed")

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bardt-large-cnn-dialoguesum-booksum-lora",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=cfg.batch_size,
    per_device_eval_batch_size=cfg.batch_size,
    weight_decay=0.01,
    save_total_limit=cfg.save_limits,
    num_train_epochs=cfg.epochs,
    save_strategy='epoch',
    predict_with_generate=True,
    load_best_model_at_end=True,
    push_to_hub=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.819100,2.759991,0.313900,0.070900,0.184300,0.184300,117.092700
